In [1]:
import matplotlib.pyplot as plt
from seaborn import heatmap
from seaborn import pairplot
from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import RangeParameter, FieldSelector, FilterSelector, Parameter, expand_plist, wwr, CategoryParameter, GenericSelector
from besos.problem import EPProblem
import pandas as pd

In [2]:
idf_file = '../IDF/v4.idf'
epw_file = '../weather/my_epw_new.epw'
building = ef.get_building(idf_file)
output_dir = './opt_out'

In [3]:
[materials.Name for materials in building.idfobjects["MATERIAL"]]# get a list of the Name property of all IDF objects of class MATERIAL
# for materials in building.idfobjects["MATERIAL"]:
#     if materials.Name =='XPS Extruded Polystyrene  - CO2 Blowing_.0795':
#         print(materials)

['Gypsum Plastering_.02',
 'Brick_.2',
 'Masonry - mediumweight Dry_.2',
 'Mortar_.03',
 'Ceramic/porcelain_.01',
 'Cement/plaster/mortar - cement plaster_.015',
 'Masonry - heavyweight Dry_.5',
 'Gypsum Plastering_.015',
 'Painted Oak_.035',
 'Masonry - heavyweight Dry_.12',
 'Masonry - heavyweight Dry_.09',
 'Brickwork Outer_.1',
 'XPS Extruded Polystyrene  - CO2 Blowing_.0795',
 'Concrete Block (Medium)_.1',
 'Gypsum Plastering_.013',
 'Masonry - heavyweight Dry_.27']

In [4]:
print(building.idfobjects["Lights"])

[
Lights,
    MainXground:Zone1 General lighting,    !- Name
    MainXground:Zone1,        !- Zone or ZoneList Name
    Dwell_DomBath_Light,      !- Schedule Name
    Watts/Area,               !- Design Level Calculation Method
    ,                         !- Lighting Level
    7.5,                      !- Watts per Zone Floor Area
    ,                         !- Watts per Person
    0,                        !- Return Air Fraction
    0.42,                     !- Fraction Radiant
    0.18,                     !- Fraction Visible
    1,                        !- Fraction Replaceable
    ELECTRIC EQUIPMENT#MainXground:Zone1#GeneralLights;    !- EndUse Subcategory
, 
Lights,
    MainXground:Zone5 General lighting,    !- Name
    MainXground:Zone5,        !- Zone or ZoneList Name
    Dwell_DomCirculation_Light,    !- Schedule Name
    Watts/Area,               !- Design Level Calculation Method
    ,                         !- Lighting Level
    5,                        !- Watts per Zo

In [5]:
insulation = FieldSelector(class_name='Material',\
                           object_name='XPS Extruded Polystyrene  - CO2 Blowing_.0795',\
                           field_name='Thickness')

In [6]:
zero_to_one_exclusive = RangeParameter(min_val = 0.01, max_val=0.99)

In [7]:
insulation_param = Parameter(selector=insulation,
                            value_descriptors=[zero_to_one_exclusive],
                            name='Insulation Thickness')
window_to_wall = wwr(RangeParameter(0.1,0.9,name="Window to Wall Ratio"))
window_to_wall

Parameter(selector=wwrSelector(), value_descriptors=[RangeParameter(name='Window to Wall Ratio', min=0.1, max=0.9)])

In [8]:
parameters = [insulation_param] 

In [9]:
objectives = ['Electricity:Facility'] # these get made into `MeterReader` or `VariableReader`
problem=EPProblem(parameters, objectives) # problem = parameters + objectives
problem

EPProblem(inputs=[Parameter(selector=FieldSelector(field_name='Thickness', class_name='Material', object_name='XPS Extruded Polystyrene  - CO2 Blowing_.0795'), value_descriptors=[RangeParameter(min=0.01, max=0.99)])], outputs=[MeterReader(class_name='Output:Meter', func=<function sum_values at 0x7f3a44e92a60>, key_name='Electricity:Facility')], minimize_outputs=[True], converters={'outputs': <class 'besos.objectives.MeterReader'>, 'constraints': <class 'besos.objectives.MeterReader'>})

In [10]:
samples = sampling.dist_sampler(sampling.lhs, problem, num_samples=20)

#samples = pd.read_csv(filename+'.csv').drop(['Electricity:Facility','DistrictCooling:Facility'], axis=1)
samples

,Insulation Thickness
0,0.362430
1,0.626649
2,0.157363
3,0.263463
4,0.785893
5,0.343237
6,0.679683
7,0.714100
8,0.227470
9,0.848442


In [11]:
evaluator = EvaluatorEP(problem, building, out_dir=output_dir, err_dir=output_dir ,epw=epw_file) # evaluator = problem + building
outputs = evaluator.df_apply(samples, keep_input=True)
outputs.describe()

Executing:   0%|          | 0/20 [00:00<?, ?row/s]

EnergyPlus Starting
EnergyPlus, Version 9.0.1-bb7ca4f0da, YMD=2022.02.04 02:04
Initializing Response Factors
Calculating CTFs for "INTERNAL FLOOR 46CM", Construction # 3
Calculating CTFs for "INTERNAL FLOOR 46CM", Construction # 3
Calculating CTFs for "INTERNAL UNINSULATED WALL 53CM", Construction # 7
Calculating CTFs for "INTERNAL UNINSULATED WALL 53CM", Construction # 7
Calculating CTFs for "PROJECT INTERNAL DOOR", Construction # 9
Calculating CTFs for "INTERNAL UNINSULATED WALL 15CM", Construction # 13
Calculating CTFs for "INTERNAL UNINSULATED WALL 12CM", Construction # 17
Calculating CTFs for "PROJECT SEMI-EXPOSED WALL", Construction # 19
Calculating CTFs for "PROJECT SEMI-EXPOSED WALL", Construction # 19
Calculating CTFs for "UNINSULATED WALL 30CM", Construction # 23
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorpti

EnergyPlus Completed Successfully.
Executing:   5%|▌         | 1/20 [01:05<20:50, 65.81s/row]

EnergyPlus Starting
EnergyPlus, Version 9.0.1-bb7ca4f0da, YMD=2022.02.04 02:05
Initializing Response Factors
Calculating CTFs for "INTERNAL FLOOR 46CM", Construction # 3
Calculating CTFs for "INTERNAL FLOOR 46CM", Construction # 3
Calculating CTFs for "INTERNAL UNINSULATED WALL 53CM", Construction # 7
Calculating CTFs for "INTERNAL UNINSULATED WALL 53CM", Construction # 7
Calculating CTFs for "PROJECT INTERNAL DOOR", Construction # 9
Calculating CTFs for "INTERNAL UNINSULATED WALL 15CM", Construction # 13
Calculating CTFs for "INTERNAL UNINSULATED WALL 12CM", Construction # 17
Calculating CTFs for "PROJECT SEMI-EXPOSED WALL", Construction # 19
Calculating CTFs for "PROJECT SEMI-EXPOSED WALL", Construction # 19
Calculating CTFs for "UNINSULATED WALL 30CM", Construction # 23
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorpti

EnergyPlus Completed Successfully.
Executing:  10%|█         | 2/20 [02:07<19:04, 63.58s/row]

...

EnergyPlus Starting
EnergyPlus, Version 9.0.1-bb7ca4f0da, YMD=2022.02.04 02:22
Initializing Response Factors
Calculating CTFs for "INTERNAL FLOOR 46CM", Construction # 3
Calculating CTFs for "INTERNAL FLOOR 46CM", Construction # 3
Calculating CTFs for "INTERNAL UNINSULATED WALL 53CM", Construction # 7
Calculating CTFs for "INTERNAL UNINSULATED WALL 53CM", Construction # 7
Calculating CTFs for "PROJECT INTERNAL DOOR", Construction # 9
Calculating CTFs for "INTERNAL UNINSULATED WALL 15CM", Construction # 13
Calculating CTFs for "INTERNAL UNINSULATED WALL 12CM", Construction # 17
Calculating CTFs for "PROJECT SEMI-EXPOSED WALL", Construction # 19
Calculating CTFs for "UNINSULATED WALL 30CM", Construction # 23
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade

EnergyPlus Completed Successfully.
Executing:  95%|█████████▌| 19/20 [19:17<01:00, 60.20s/row]

EnergyPlus Starting
EnergyPlus, Version 9.0.1-bb7ca4f0da, YMD=2022.02.04 02:23
Initializing Response Factors
Calculating CTFs for "INTERNAL FLOOR 46CM", Construction # 3
Calculating CTFs for "INTERNAL FLOOR 46CM", Construction # 3
Calculating CTFs for "INTERNAL UNINSULATED WALL 53CM", Construction # 7
Calculating CTFs for "INTERNAL UNINSULATED WALL 53CM", Construction # 7
Calculating CTFs for "PROJECT INTERNAL DOOR", Construction # 9
Calculating CTFs for "INTERNAL UNINSULATED WALL 15CM", Construction # 13
Calculating CTFs for "INTERNAL UNINSULATED WALL 12CM", Construction # 17
Calculating CTFs for "PROJECT SEMI-EXPOSED WALL", Construction # 19
Calculating CTFs for "UNINSULATED WALL 30CM", Construction # 23
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade

EnergyPlus Completed Successfully.
Executing: 100%|██████████| 20/20 [20:18<00:00, 60.93s/row]


,Insulation Thickness,Electricity:Facility
count,20.000000,2.000000e+01
mean,0.500760,9.842877e+09
std,0.291212,0.000000e+00
min,0.037779,9.842877e+09
25%,0.254464,9.842877e+09
50%,0.509748,9.842877e+09
75%,0.732048,9.842877e+09
max,0.958687,9.842877e+09
